In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras import models
from keras import layers
from keras.layers import BatchNormalization
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras import models
from keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import os
import seaborn as sns
from keras.regularizers import l2
from tensorflow.keras import backend as K

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
epochs = 50
batch_size = 64
img_rows, img_cols = 48, 48
classes = ['angry', 'disgust','fear','happy','neutral','sad','surprise']
# classes = ['disgust', 'fear', 'happy','pain', 'sad']

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

def get_data(batch_size, target_size, train_path, test_path):
    train_datagen = ImageDataGenerator(
        validation_split=0.2,
        rescale=1./255,
        featurewise_center=False,
        featurewise_std_normalization=False,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=lambda img: cv2.resize(img, (224,224), interpolation=cv2.INTER_LINEAR)
        

    )

    validation_datagen = ImageDataGenerator(
        validation_split=0.2,
        rescale=1./255
    )

    train_generator = train_datagen.flow_from_directory(
        train_path,
        batch_size=batch_size,
        target_size=target_size,
        color_mode='rgb',
        shuffle=True,
        class_mode='categorical',
        subset='training'
        
        
    )  # Set as training data


    validation_generator = validation_datagen.flow_from_directory(
        test_path,
        batch_size=batch_size,
        target_size=target_size,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical',
        subset='validation'
        
    )  # Set as validation data

    return train_generator, train_generator.labels, validation_generator, validation_generator.labels
# function to plot the confusion matrix
def plot_confusion_matrix(classes, model, model_name):
    batch_size=32
    num_of_test_samples = 4941
    target_names =classes
    #Confution Matrix and Classification Report
    Y_pred = model.predict(validation_generator, num_of_test_samples // batch_size)
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    cm = confusion_matrix(validation_generator.classes, y_pred)
    print(cm)
    print('Classification Report')
    print(classification_report(validation_generator.classes, y_pred, target_names=target_names))
    # Normalise
    cmn = cm.astype('float') / cm.sum(axis=1)
    fig, ax = plt.subplots(figsize=(20,7))

    sns.heatmap(cmn, center=0, annot=True, fmt='.2f', linewidths=1,  xticklabels=target_names, yticklabels=target_names)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('cofusion_matrix of **' + f'( {model_name} )**')
   
    plt.show(block=False)
    plt.show()


#function to graph the accuracy and loss
def plot_graph(history, model_name):
    f, ax = plt.subplots()
    ax.plot([None] + history.history['accuracy'], 'o-')
    ax.plot([None] + history.history['val_accuracy'], 'x-')
    # Plot legend and use the best location automatically: loc = 0.
    ax.legend(['Train acc', 'Validation acc'], loc = 0)
    ax.set_title('Training/Validation acc per Epoch '+f'({model_name})')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')

 
    plt.show()

    f, ax = plt.subplots()
    ax.plot([None] + history.history['loss'], 'o-')
    ax.plot([None] + history.history['val_loss'], 'x-')

    # Plot legend and use the best location automatically: loc = 0.
    ax.legend(['Train loss', "Val loss"], loc = 1)
    ax.set_title('Training/Validation Loss per Epoch '+f'({model_name})')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
   
    plt.show()



In [4]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D,Dense,Reshape, Add, Activation,Permute,Lambda,Concatenate, Conv2D,multiply,Layer


class CBAMBlock(keras.layers.Layer):
    def __init__(self, ratio=64,name='cbamblock'):
        super().__init__(name=name)
        self.ratio = ratio

    @tf.function
    def cbam_block(self, cbam_feature):
        cbam_feature = self.channel_attention(cbam_feature)
        cbam_feature = self.spatial_attention(cbam_feature)
        return cbam_feature

    def channel_attention(self, input_feature):
        channel_axis = 1 if tf.keras.backend.image_data_format() == "channels_first" else -1
        channel = input_feature.shape[channel_axis]

        shared_layer_one = Dense(
            channel // self.ratio,
            activation='relu',
            kernel_initializer='he_normal',
            use_bias=True,
            bias_initializer='zeros',
        )
        shared_layer_two = Dense(
            channel,
            kernel_initializer='he_normal',
            use_bias=True,
            bias_initializer='zeros',
        )

        avg_pool = GlobalAveragePooling2D()(input_feature)
        avg_pool = Reshape((1, 1, channel))(avg_pool)
        assert avg_pool.shape[1:] == (1, 1, channel)
        avg_pool = shared_layer_one(avg_pool)
        assert avg_pool.shape[1:] == (1, 1, channel // self.ratio)
        avg_pool = shared_layer_two(avg_pool)
        assert avg_pool.shape[1:] == (1, 1, channel)

        max_pool = GlobalMaxPooling2D()(input_feature)
        max_pool = Reshape((1, 1, channel))(max_pool)
        assert max_pool.shape[1:] == (1, 1, channel)
        max_pool = shared_layer_one(max_pool)
        assert max_pool.shape[1:] == (1, 1, channel // self.ratio)
        max_pool = shared_layer_two(max_pool)
        assert max_pool.shape[1:] == (1, 1, channel)

        cbam_feature = Add()([avg_pool, max_pool])
        cbam_feature = Activation('sigmoid')(cbam_feature)

        if tf.keras.backend.image_data_format() == "channels_first":
            cbam_feature = Permute((3, 1, 2))(cbam_feature)

        return multiply([input_feature, cbam_feature])

    def spatial_attention(self, input_feature):
        kernel_size = 7

        if tf.keras.backend.image_data_format() == "channels_first":
            channel = input_feature.shape[1]
            cbam_feature = Permute((2, 3, 1))(input_feature)
        else:
            channel = input_feature.shape[-1]
            cbam_feature = input_feature

        avg_pool = Lambda(lambda x: tf.reduce_mean(x, axis=3, keepdims=True))(cbam_feature)
        assert avg_pool.shape[-1] == 1
        max_pool = Lambda(lambda x: tf.reduce_max(x, axis=3, keepdims=True))(cbam_feature)
        assert max_pool.shape[-1] == 1
        concat = Concatenate(axis=3)([avg_pool, max_pool])
        assert concat.shape[-1] == 2
        cbam_feature = Conv2D(
            filters=1,
            kernel_size=kernel_size,
            strides=1,
            padding='same',
            activation='sigmoid',
            kernel_initializer='he_normal',
            use_bias=False,
        )(concat)
        assert cbam_feature.shape[-1] == 1

        if tf.keras.backend.image_data_format() == "channels_first":
            cbam_feature = Permute((3, 1, 2))(cbam_feature)

        return multiply([input_feature, cbam_feature])


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply

def se_block(inputs, ratio=4, pooling_type='avg'):
    filters = inputs.shape[-1]
    se_shape = (1, 1, filters)
    
    se = GlobalAveragePooling2D()(inputs)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='hard_sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    return multiply([inputs, se])


In [6]:
tf.keras.utils.custom_object_scope({'SeBlock':se_block})

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, GlobalAveragePooling2D, Dense, BatchNormalization

from keras.models import Model
# Input layer
input_shape = (224, 224, 3)
input_layer = Input(shape=input_shape)

# Convolutional layers
x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(input_layer) #conv/s2

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)#conv dw/s1
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x) #conv/s1
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same', activation='relu')(x) #conv dw/s2
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x) # conv/s1
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same', activation='relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

#cbam

#cbam=CBAMBlock()(x)

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x) #conv dw/s2
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same', activation='relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

cbam=CBAMBlock(name='cbamblock_1')(x)

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(cbam)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)



x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

#cbam=CBAMBlock(name='cbamblock_2')(x)

x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same', activation='relu')(x)
x = Conv2D(1024, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)


x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = Conv2D(1024, (1, 1), strides=(1, 1), padding='same', activation=tf.nn.relu6)(x)
x = BatchNormalization()(x)

#cbam=CBAMBlock(name='cbamblock_3')(x)

x=GlobalAveragePooling2D()(x)

x=Dense(1024, activation='relu')(x)

out= Dense(7, activation='softmax')(x)

# Create the Patch Extraction Block model
model = Model(input_layer, out)



# Print a summary of the model architecture
model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 depthwise_conv2d (Depthwise  (None, 112, 112, 32)     320       
 Conv2D)                                                         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      2112      
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 depthwise_conv2d_1 (Depthwi  (None, 56, 56, 64)       640   

In [6]:
train_path='/kaggle/input/newfer/train/'
test_path='/kaggle/input/newfer/test/'
train_generator,train_lbl,validation_generator,lbl= get_data(batch_size=64, target_size=(224,224), train_path=train_path,test_path=test_path)


Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [7]:
# adding class weights
import numpy as np
from sklearn.utils import class_weight



# Calculate the class weights

class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(lbl), y=lbl)
# Convert the class weights to a dictionary
class_weights_dict = dict(enumerate(class_weights))





In [9]:
print(class_weights_dict)

{0: 0.8969202898550724, 1: 1.209035409035409, 2: 0.6866851595006935, 3: 1.5970967741935485, 4: 1.0250517598343685}


In [11]:
import tensorflow as tf
from math import pi
import keras
from tensorflow.keras.losses import Loss, CategoricalCrossentropy
from tensorflow.keras import initializers
import keras.backend as K

class CombinedLoss(Loss):
    def __init__(self, lambda_center=0.01, lambda_=0.001, margin=0.35, scale=64,num_classes=7,feature_dim=1024):
        super(CombinedLoss, self).__init__()
        self.lambda_center = lambda_center
        self.lambda_ = lambda_
        self.margin = margin
        self.scale = scale
        self.threshold = tf.math.cos(pi - margin)
        self.cos_m = tf.math.cos(margin)
        self.sin_m = tf.math.sin(margin)
        self.safe_margin = self.sin_m * margin
        self.feature_dim = feature_dim  # Update feature_dim to 1024
        self.centers = K.variable(initializers.glorot_uniform()((feature_dim,num_classes)))
        
    @tf.function
    def call(self, y_true, y_pred):
        # CenterLoss
        softmax_loss = CategoricalCrossentropy()(y_true, y_pred)
        centers_batch = tf.gather(self.centers, tf.argmax(y_true, axis=1))
        center_loss = softmax_loss + 0.01 * (0.5 * self.lambda_center * tf.reduce_sum(tf.square(y_pred - centers_batch)))

        # ArcLoss
        cos_t = y_pred
        sin_t = tf.math.sqrt(1 - tf.math.square(cos_t))
        cos_t_margin = tf.where(cos_t > self.threshold,
                                cos_t * self.cos_m - sin_t * self.sin_m,
                                cos_t - self.safe_margin)
        mask = y_true
        cos_t_onehot = cos_t * mask
        cos_t_margin_onehot = cos_t_margin * mask
        logits = (cos_t + cos_t_margin_onehot - cos_t_onehot) * self.scale
        arc_loss = tf.nn.softmax_cross_entropy_with_logits(y_true, logits)

        # Combined Loss
        loss = self.lambda_ * center_loss + self.lambda_ * arc_loss
        return loss

    

    def get_config(self):
        config = super(CombinedLoss, self).get_config()
        config.pop('reduction', None)
        config.pop('name', None)
        config.update({"lambda_center": self.lambda_center, 
                       "lambda_": self.lambda_, 
                       "margin": self.margin,
                       "scale": self.scale
                      })
        return config


In [12]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau

initial_learning_rate = 0.01
batch_size = 128
weight_decay = 0.0001

opt = SGD(learning_rate=initial_learning_rate,weight_decay=weight_decay,momentum=0.9)
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=10, monitor='val_accuracy', mode='max')



model.compile(optimizer = opt , loss=CombinedLoss(), metrics = ['accuracy'])


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.image import ImageDataGenerator
epochs = 100  # will increase late it was 300 n the code
history = model.fit(train_generator,
                         epochs=epochs,
                         #steps_per_epoch=620,
                         validation_data=validation_generator,
                         class_weight=class_weights_dict,
                         callbacks=[lr_scheduler])

feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)  # Output before the final dense layer
train_features = feature_extractor.predict(train_generator, verbose=1)
val_features = feature_extractor.predict(validation_generator, verbose=1)

# Assuming train_lbl and lbl are in the correct format for RandomForestClassifier
# Train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_features, train_lbl)

# Predict and evaluate the model using validation data
val_predictions = rf_classifier.predict(val_features)
val_accuracy = accuracy_score(lbl, val_predictions)
print(f"Validation accuracy: {val_accuracy * 100:.2f}%")

# Visualize the model performance
plot_graph(history=history, model_name='CNN Model')
plot_confusion_matrix(classes=classes, model=rf_classifier, model_name='Random Forest')

#visualizing the model 
plot_graph(history=history, model_name='Model')
#confusion matrix of the model
plot_confusion_matrix(classes=classes, model=model, model_name='Model')
